# Business Impact, Guardrails & Advanced Analysis

This notebook covers tools for translating test results into business decisions:
- Revenue impact projections
- Guardrail metric monitoring
- Segment analysis
- Survival/timing analysis

In [1]:
from pyexpstats import business, segments, timing, conversion

## Revenue Impact Projections

In [2]:
# First, run a conversion analysis to get results
result = conversion.analyze(
    control_visitors=8000,
    control_conversions=400,
    variant_visitors=8000,
    variant_conversions=456,
)

# Project the business impact of shipping the variant
impact = business.project_impact(
    control_rate=result.control_rate,
    variant_rate=result.variant_rate,
    lift_percent=result.lift_percent,
    lift_ci_lower=result.confidence_interval_lower,
    lift_ci_upper=result.confidence_interval_upper,
    monthly_visitors=200000,
    revenue_per_conversion=65.0,
    cost_of_change=5000,
)

print(f"Monthly revenue lift:  ${impact.monthly_revenue_lift:,.0f}")
print(f"Annual revenue lift:   ${impact.annual_revenue_lift:,.0f}")
print(f"Revenue range (annual): ${impact.revenue_lift_range[0]:,.0f} to ${impact.revenue_lift_range[1]:,.0f}")
print(f"\nAdditional conversions/month: {impact.monthly_additional_conversions:,.0f}")
print(f"Additional conversions/year:  {impact.annual_additional_conversions:,.0f}")
print(f"\nValue per visitor (control):  ${impact.value_per_visitor_control:.4f}")
print(f"Value per visitor (variant):  ${impact.value_per_visitor_variant:.4f}")
print(f"Incremental value/visitor:    ${impact.incremental_value_per_visitor:.4f}")

Monthly revenue lift:  $91,000
Annual revenue lift:   $1,092,000
Revenue range (annual): $2 to $1,090

Additional conversions/month: 1,400
Additional conversions/year:  16,800

Value per visitor (control):  $3.2500
Value per visitor (variant):  $3.7050
Incremental value/visitor:    $0.4550


In [3]:
print(f"P(positive impact): {impact.probability_positive_impact:.1%}")
print(f"P(negative impact): {impact.probability_negative_impact:.1%}")
print(f"Expected value:     ${impact.expected_value:,.0f}")

P(positive impact): 100.0%
P(negative impact): 0.0%
Expected value:     $1,092,000


## Guardrail Metrics

In [4]:
# Monitor guardrail metrics to ensure the variant doesn't hurt key metrics
guardrails = business.check_guardrails(
    guardrails=[
        {
            "name": "Page Load Time",
            "metric_type": "mean",
            "direction": "increase_is_bad",
            "threshold_percent": 5.0,
            "critical_threshold_percent": 10.0,
            "control_data": [2.1, 2.3, 1.9, 2.5, 2.0, 2.2, 2.4, 1.8, 2.3, 2.1] * 50,
            "variant_data": [2.2, 2.4, 2.0, 2.5, 2.1, 2.3, 2.5, 1.9, 2.4, 2.2] * 50,
        },
        {
            "name": "Bounce Rate",
            "metric_type": "proportion",
            "direction": "increase_is_bad",
            "threshold_percent": 5.0,
            "control_data": {"count": 2400, "total": 8000},
            "variant_data": {"count": 2480, "total": 8000},
        },
        {
            "name": "Revenue per Session",
            "metric_type": "mean",
            "direction": "decrease_is_bad",
            "threshold_percent": 3.0,
            "control_data": [3.2, 0, 0, 5.1, 0, 0, 4.8, 0, 0, 6.2] * 50,
            "variant_data": [3.5, 0, 0, 5.3, 0, 0, 5.0, 0, 0, 6.0] * 50,
        },
    ]
)

print(f"All passed: {guardrails.all_passed}")
print(f"Can ship: {guardrails.can_ship}")
print(f"\nResults:")
for r in guardrails.results:
    print(f"  [{r.status.upper():8s}] {r.name}: {r.change_percent:+.1f}% change (p={r.p_value:.4f})")

All passed: True
Can ship: True

Results:
  [PASSED  ] Page Load Time: +4.2% change (p=0.0000)
  [PASSED  ] Bounce Rate: +3.3% change (p=0.1749)
  [PASSED  ] Revenue per Session: +2.6% change (p=0.7498)


In [5]:
if guardrails.passed:
    print("Passed:", ", ".join(guardrails.passed))
if guardrails.warnings:
    print("Warnings:", ", ".join(guardrails.warnings))
if guardrails.failures:
    print("Failures:", ", ".join(guardrails.failures))

Passed: Page Load Time, Bounce Rate, Revenue per Session


## Segment Analysis

In [6]:
# Analyze how the treatment effect varies across segments
seg_report = segments.analyze_segments(
    segments_data=[
        {"segment_name": "device", "segment_value": "desktop",
         "control_visitors": 3000, "control_conversions": 150,
         "variant_visitors": 3000, "variant_conversions": 180},
        {"segment_name": "device", "segment_value": "mobile",
         "control_visitors": 4000, "control_conversions": 200,
         "variant_visitors": 4000, "variant_conversions": 260},
        {"segment_name": "device", "segment_value": "tablet",
         "control_visitors": 1000, "control_conversions": 50,
         "variant_visitors": 1000, "variant_conversions": 48},
        {"segment_name": "traffic_source", "segment_value": "organic",
         "control_visitors": 3500, "control_conversions": 175,
         "variant_visitors": 3500, "variant_conversions": 210},
        {"segment_name": "traffic_source", "segment_value": "paid",
         "control_visitors": 4500, "control_conversions": 225,
         "variant_visitors": 4500, "variant_conversions": 278},
    ],
    confidence=95,
    correction_method="bonferroni",
)

print(f"Overall lift: {seg_report.overall_lift:.1f}%")
print(f"Overall significant: {seg_report.overall_is_significant}")
print(f"Heterogeneity detected: {seg_report.heterogeneity_detected}")
print(f"Simpson's paradox risk: {seg_report.simpsons_paradox_risk}")
print(f"\nBest segment: {seg_report.best_segment}")
print(f"Worst segment: {seg_report.worst_segment}")

Overall lift: 22.0%
Overall significant: True
Heterogeneity detected: True
Simpson's paradox risk: False

Best segment: device=mobile
Worst segment: device=mobile


In [7]:
print(f"\nSegment details:")
for seg in seg_report.segments:
    sig_marker = "*" if seg.is_significant else " "
    adequate = "ok" if seg.sample_size_adequate else "LOW"
    print(f"  {seg.segment_name}/{seg.segment_value}: "
          f"lift={seg.lift_percent:+.1f}% "
          f"CI=[{seg.lift_ci_lower:+.1f}%, {seg.lift_ci_upper:+.1f}%] "
          f"p={seg.p_value:.4f} {sig_marker} ({adequate})")


Segment details:
  device/desktop: lift=+20.0% CI=[-3.1%, +43.1%] p=0.1005   (ok)
  device/mobile: lift=+30.0% CI=[+9.6%, +50.4%] p=0.0046 * (ok)
  device/tablet: lift=-4.0% CI=[-41.8%, +33.8%] p=0.9175   (ok)
  traffic_source/organic: lift=+20.0% CI=[-1.4%, +41.4%] p=0.0747   (ok)
  traffic_source/paid: lift=+23.6% CI=[+4.6%, +42.5%] p=0.0170   (ok)


## Survival / Timing Analysis

In [8]:
# Analyze time-to-event data (e.g., time to first purchase)
import random
random.seed(42)

# Simulate time-to-purchase data
control_times = [random.expovariate(1/7) for _ in range(200)]
control_events = [1 if t < 30 else 0 for t in control_times]  # Censored at 30 days

treatment_times = [random.expovariate(1/5.5) for _ in range(200)]
treatment_events = [1 if t < 30 else 0 for t in treatment_times]

timing_result = timing.analyze(
    control_times=control_times,
    control_events=control_events,
    treatment_times=treatment_times,
    treatment_events=treatment_events,
)

print(f"Control median time:   {timing_result.control_median_time:.1f} days")
print(f"Treatment median time: {timing_result.treatment_median_time:.1f} days")
print(f"Hazard ratio:          {timing_result.hazard_ratio:.3f}")
print(f"HR 95% CI:             [{timing_result.hazard_ratio_ci_lower:.3f}, {timing_result.hazard_ratio_ci_upper:.3f}]")
print(f"Significant:           {timing_result.is_significant}")
print(f"P-value:               {timing_result.p_value:.4f}")

Control median time:   4.4 days
Treatment median time: 4.3 days
Hazard ratio:          1.150
HR 95% CI:             [0.944, 1.402]
Significant:           False
P-value:               0.3279


In [9]:
# Survival curve for the control group
curve = timing.survival_curve(
    times=control_times,
    events=control_events,
)

print(f"Median survival time: {curve.median_time:.1f} days")
print(f"Total: {curve.total}, Events: {curve.events}, Censored: {curve.censored}")
print(f"\nSurvival probabilities at key time points:")
for i in range(0, len(curve.times), max(1, len(curve.times) // 8)):
    print(f"  Day {curve.times[i]:5.1f}: {curve.survival_probabilities[i]:.1%} "
          f"[{curve.confidence_lower[i]:.1%}, {curve.confidence_upper[i]:.1%}]")

Median survival time: 4.4 days
Total: 200, Events: 195, Censored: 5

Survival probabilities at key time points:
  Day   0.0: 100.0% [100.0%, 100.0%]
  Day   0.7: 88.0% [82.6%, 91.8%]
  Day   1.8: 76.0% [69.5%, 81.3%]
  Day   2.6: 64.0% [56.9%, 70.2%]
  Day   4.1: 52.0% [44.9%, 58.7%]
  Day   5.9: 40.0% [33.2%, 46.7%]
  Day   8.1: 28.0% [22.0%, 34.3%]
  Day  13.2: 16.0% [11.3%, 21.4%]
  Day  22.1: 4.0% [1.9%, 7.4%]


In [10]:
# Event rate comparison
rates = timing.analyze_rates(
    control_events=150,
    control_exposure=1000,     # 1000 person-days
    treatment_events=180,
    treatment_exposure=1000,
)

print(f"Control rate:      {rates.control_rate:.3f} events/day")
print(f"Treatment rate:    {rates.treatment_rate:.3f} events/day")
print(f"Rate ratio:        {rates.rate_ratio:.3f}")
print(f"Rate difference:   {rates.rate_difference_percent:+.1f}%")
print(f"Significant:       {rates.is_significant}")

Control rate:      0.150 events/day
Treatment rate:    0.180 events/day
Rate ratio:        1.200
Rate difference:   +20.0%
Significant:       False


In [11]:
print(timing.summarize(timing_result, test_name="Time-to-Purchase Test"))

## ⏱️ Time-to-Purchase Test Results

### ⚠️ No Significant Timing Effect

**The treatment does not significantly affect event timing.**

### 📈 Key Metrics

| Metric | Control | Treatment |
|--------|---------|-----------|
| Median time | 4.4 | 4.3 |
| Events | 195 | 198 |
| Censored | 5 | 2 |

- **Hazard ratio:** 1.150 (95% CI: 0.944 - 1.402)
- **P-value:** 0.3279
- **Time saved:** 0.1 units (2.4% faster)

### 📝 Interpretation

**Insufficient evidence** to conclude there is a timing effect (p > 0.10).
